In [78]:
import drawSvg as draw
import random

class Extent():
    
    def __init__(self, x=0, y=0, w=0, h=0):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        
    def __repr__(self):
        return f"{self.x},{self.y},{self.w},{self.h}"
        
    def __iter__(self):
        yield (self.x, self.y, self.w, self.h)
        
    def draw(self, drawing, offset=0, **config):
        rect = draw.Rectangle(
            self.x+offset, 
            self.y+offset, 
            self.w-(offset*2), 
            self.h-(offset*2),
            **config)
        drawing.append(rect)
        
    
    @property
    def x_mid(self):
        return self.w / 2 + self.x
    
    @property
    def y_mid(self):
        return self.h / 2 + self.y
        
        
"""
(0,200) (50,200) (100,200) (150,200) (200,200)
(0,150) (50,150) (100,150) (150,150) (200,150)
(0,100) (50,100) (100,100) (150,100) (200,100)
(0,50 ) (50,50 ) (100,50 ) (150,50 ) (200,50 )
(0,0  ) (50,0  ) (100,0. ) (150,0  ) (200,0. )
"""

class VisualQuadTree():

    def __init__(
        self, 
        top_left=None, 
        top_right=None, 
        bot_left=None,
        bot_right=None, 
        extent=None, 
        depth=0,
        max_depth=3):
        
        self.extent = extent
        self.depth = depth
        self.max_depth = max_depth
        
        
        if self.depth < max_depth:
            
            e = extent
            
            threshold = self.depth/float(self.max_depth)
            if random.random() > threshold:
                top_left_extent  = Extent(e.x, e.y_mid, e.w/2, e.h/2)
            else:
                top_left_extent  = e
            
            if random.random() > threshold:
                top_right_extent = Extent(e.x_mid, e.y_mid, e.w/2, e.h/2)
            else:
                top_right_extent = e
                
            if random.random() > threshold:
                bot_left_extent  = Extent(e.x, e.y, e.w/2, e.h/2)
            else:
                bot_left_extent  = e
                
            if random.random() > threshold:
                bot_right_extent = Extent(e.x_mid, e.y, e.w/2, e.h/2)
            else:
                bot_right_extent = e
            
            self.top_left    = VisualQuadTree(extent=top_left_extent, depth=self.depth+1, max_depth=self.max_depth)
            self.top_right   = VisualQuadTree(extent=top_right_extent, depth=self.depth+1, max_depth=self.max_depth)
            self.bot_left    = VisualQuadTree(extent=bot_left_extent, depth=self.depth+1, max_depth=self.max_depth)
            self.bot_right   = VisualQuadTree(extent=bot_right_extent, depth=self.depth+1, max_depth=self.max_depth)
            
            
        else:
            self.top_left  = None
            self.top_right = None
            self.bot_left  = None
            self.bot_right = None

    def draw(self, drawing, name="", **config):
        # print("\t"*self.depth, name, self.depth, self.extent)
        config = dict(
                stroke=["red", "green", "purple", "orange", "blue", "teal", "yellow", "magenta"][self.depth],
                stroke_width=1,
                fill_opacity=0,
            )
        self.extent.draw(drawing, offset=self.depth*3, **config)
        # self.extent.draw(drawing, offset=0, **config)
        for quad_name, quadrant in {
            "tl":self.top_left, 
            "tr":self.top_right, 
            "bl":self.bot_left,
            "br":self.bot_right, 
        }.items():
            if quadrant != None:
                # print(quadrant.depth)
                quadrant.draw(drawing, name=quad_name, **config)
        

drawing = draw.Drawing(801, 801, origin=(0,0), displayInline=False)
# bounding_rect = draw.Rectangle(
#     0,
#     0,
#     drawing.width, 
#     drawing.height,
#     fill="grey",
#     fill_opacity=1.0
#     )
# drawing.append(bounding_rect)

extent = Extent(0, 0, drawing.width-1, drawing.height-1)
tree = VisualQuadTree(extent=extent, max_depth=7)
tree.draw(drawing)
drawing.savePng("tree.png")
drawing

In [181]:
from pprint import pformat

class Node():
    def __init__(self, left=None, right=None, depth=0, max_depth=3):
        self.left = left
        self.right = right
        self.depth = depth
        
        if self.depth != max_depth:
            self.left = Node(depth=self.depth+1)
            self.right = Node(depth=self.depth+1)
    
    def __repr__(self):
        return pformat(self.__dict__)
    
Tree(3)

{'root': {'depth': 0,
 'left': {'depth': 1,
 'left': {'depth': 2,
 'left': {'depth': 3, 'left': None, 'right': None},
 'right': {'depth': 3, 'left': None, 'right': None}},
 'right': {'depth': 2,
 'left': {'depth': 3, 'left': None, 'right': None},
 'right': {'depth': 3, 'left': None, 'right': None}}},
 'right': {'depth': 1,
 'left': {'depth': 2,
 'left': {'depth': 3, 'left': None, 'right': None},
 'right': {'depth': 3, 'left': None, 'right': None}},
 'right': {'depth': 2,
 'left': {'depth': 3, 'left': None, 'right': None},
 'right': {'depth': 3, 'left': None, 'right': None}}}}}